In [1]:
%cd /content/drive/MyDrive/요약 데이터

/content/drive/MyDrive/요약 데이터


In [2]:
%ls

'1.문서요약 텍스트'/  '4.논문자료 요약'/                     document.csv
'2.도서자료 요약'/    'data_json2pd(논문,문서,도서).ipynb'   paper.csv


In [3]:
import os 
import io
import pandas as pd
import json

# 4.논문자료 요약

In [4]:
class JSONDataProcessor:
    def __init__(self, root_folder):
        """
        JSONDataProcessor 클래스를 초기화합니다.

        Args:
            root_folder (str): JSON 파일들이 포함된 최상위 폴더의 경로.
        """
        self.root_folder = root_folder

    def combine_json_files(self):
        """
        주어진 폴더 안에 있는 모든 JSON 파일들을 읽어들여 데이터프레임으로 결합합니다.

        Returns:
            pandas.DataFrame: JSON 파일들을 결합한 결과로 생성된 데이터프레임.
        """
        dataframes = []
        for root, dirs, files in os.walk(self.root_folder):
            for file in files:
                if file.endswith('.json'):
                    file_path = os.path.join(root, file)
                    with open(file_path, 'r', encoding = 'UTF-8') as json_file:
                        json_data = json.load(json_file)
                        data = json_data['data']
                        df = pd.DataFrame(data)
                        dataframes.append(df)

        combined_df = pd.concat(dataframes, ignore_index=True)
        return combined_df

    def extract_summary_columns(self, combined_df, column_names):
        """
        데이터프레임에서 주어진 컬럼들을 분리하여 해당 컬럼의 원소를 컬럼명으로 하는 새로운 데이터프레임을 생성합니다.

        Args:
            combined_df (pandas.DataFrame): 분리를 수행할 대상 데이터프레임.
            column_names (list): 분리할 컬럼명들의 리스트.

        Returns:
            pandas.DataFrame: 분리를 수행한 결과로 생성된 데이터프레임.
        """
        for column_name in column_names:
            column_data = combined_df[column_name]

            column_data_list = []
            for item in column_data:
                if isinstance(item, list):
                    column_data_list.append(item[0])
                else:
                    column_data_list.append(item)

            new_df = pd.DataFrame(column_data_list)
            combined_df.drop(column_name, axis=1, inplace=True)
            combined_df = pd.concat([combined_df, new_df], axis=1)

        return combined_df

    def make_unique_column_names(self, df):
        """
        데이터프레임의 컬럼명이 중복된 경우 고유한 컬럼명을 만들어줍니다.

        Args:
            df (pandas.DataFrame): 중복된 컬럼명이 포함된 데이터프레임.

        Returns:
            pandas.DataFrame: 컬럼명이 중복을 피한 데이터프레임.
        """
        unique_columns = []
        for col in df.columns:
            if col in unique_columns:
                counter = 1
                new_col = f"{col}_{counter}"
                while new_col in unique_columns:
                    counter += 1
                    new_col = f"{col}_{counter}"
                unique_columns.append(new_col)
            else:
                unique_columns.append(col)

        df.columns = unique_columns
        return df

# Usage example:
folder_path = '/content/drive/MyDrive/요약 데이터/4.논문자료 요약'
data_processor = JSONDataProcessor(folder_path)

combined_dataframe = data_processor.combine_json_files()

columns_to_split = ['summary_entire', 'summary_section']
combined_dataframe = data_processor.extract_summary_columns(combined_dataframe, columns_to_split)

combined_dataframe = data_processor.make_unique_column_names(combined_dataframe)
# 두 열(original, summary)을 세로로 쌓기
stacked_column_o = pd.concat([combined_dataframe['orginal_text'], combined_dataframe['orginal_text_1']], axis=0, ignore_index=True)
stacked_column_s = pd.concat([combined_dataframe['summary_text'], combined_dataframe['summary_text_1']], axis=0, ignore_index=True)

# 쌓인 열을 사용하여 새 데이터 프레임 생성
data_논문 = pd.DataFrame({'original_text': stacked_column_o, 'summary_text': stacked_column_s})

In [5]:
data_논문.to_csv('paper.csv')

# 1.문서요약 텍스트

In [6]:
def combine_json_files_recursive(folder_path):
    data_frames = []

    # Traverse the directory recursively
    for root, dirs, files in os.walk(folder_path):
        for file_name in files:
            file_path = os.path.join(root, file_name)

            # Check if the file is a JSON file
            if file_name.endswith('.json') and os.path.isfile(file_path):
                # Read JSON file into a DataFrame
                with open(file_path, 'r') as file:
                    json_data = json.load(file)
                    df = pd.DataFrame(json_data['documents'])

                # Check for duplicate column names
                duplicates = df.columns[df.columns.duplicated()]
                for duplicate in duplicates:
                    # Add suffix to distinguish duplicate column names
                    suffix = 1
                    while duplicate + '_' + str(suffix) in df.columns:
                        suffix += 1
                    df.rename(columns={duplicate: duplicate + '_' + str(suffix)}, inplace=True)

                # Append DataFrame to the list
                data_frames.append(df)

    # Concatenate all DataFrames into a single DataFrame
    combined_df = pd.concat(data_frames, ignore_index=True)
    
    return combined_df

folder_path = '1.문서요약 텍스트'
combined_df = combine_json_files_recursive(folder_path)
data_문서 = combined_df[['text','abstractive']]

In [8]:
data_문서.to_csv('document.csv')

# 2.도서자료 요약

In [4]:
def combine_json_files_recursive(folder_path):
    data_frames = []

    # Traverse the directory recursively
    for root, dirs, files in os.walk(folder_path):
        for file_name in files:
            file_path = os.path.join(root, file_name)

            # Check if the file is a JSON file
            if file_name.endswith('.json') and os.path.isfile(file_path):
                # Read JSON file into a DataFrame
                with open(file_path, 'r') as file:
                    json_data = json.load(file)
                    print(f"File: {file_path}")
                    print(f"JSON Data: {json_data}")
                    if isinstance(json_data, dict):
                        df = pd.DataFrame([json_data])  # Convert dictionary to DataFrame
                        df = df[['passage', 'summary']]  # Select desired columns
                        data_frames.append(df)
                    else:
                        print(f"Invalid JSON data in file: {file_path}")
            
    # Concatenate all DataFrames into a single DataFrame
    combined_df = pd.concat(data_frames, ignore_index=True)
    
    return combined_df


folder_path = '2.도서자료 요약'
combined_data = combine_json_files_recursive(folder_path)

print(combined_data)


KeyboardInterrupt: ignored